In [1]:
!pip install cmake
!pip install dlib
!python -m pip install dlib-19.24.1-cp311-cp311-win_amd64.whl
!pip install face-recognition
!pip install opencv-python

Processing c:\users\lenovo\downloads\computervisionfinal-main\dlib-19.24.1-cp311-cp311-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
# importing librarys
import cv2
import numpy as npy
import face_recognition as face_rec
# function
# def resize(img, size) :
#     width = int(img.shape[1]*size)
#     height = int(img.shape[0] * size)
#     dimension = (width, height)
#     return cv2.resize(img, dimension, interpolation= cv2.INTER_AREA)


# img declaration
first = face_rec.load_image_file('images/Akyrys.jpeg')
first = cv2.cvtColor(first, cv2.COLOR_BGR2RGB)
# first = resize(first, 0.50)
second = face_rec.load_image_file('images/Bayansulu.jpeg')
# second = resize(second, 0.50)
second = cv2.cvtColor(second, cv2.COLOR_BGR2RGB)

# # finding face location
img_encoding = face_rec.face_encodings(first)[0]
img_encoding2 = face_rec.face_encodings(second)[0]

result = face_rec.compare_faces([img_encoding], img_encoding2)
print("Result: ", result)
print('hi')
cv2.putText(second, f'{result}', (10,30), cv2.FONT_HERSHEY_COMPLEX, 1,(255,255,255), 2 )

# plt.imshow(cv2.cvtColor(first, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

# plt.imshow(cv2.cvtColor(second, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

cv2.imshow('main_img', first)
cv2.imshow('test_img', second)
cv2.waitKey(0)
cv2.destroyAllWindows()

Result:  [True]
hi


In [3]:
!pip install pyttsx3

  Using cached pyttsx3-2.90-py3-none-any.whl (39 kB)
  Using cached comtypes-1.2.0-py2.py3-none-any.whl.metadata (3.7 kB)
  Using cached pypiwin32-223-py3-none-any.whl (1.7 kB)
Using cached comtypes-1.2.0-py2.py3-none-any.whl (184 kB)


In [ ]:
import cv2
from IPython.display import display, Javascript
import numpy as np
import face_recognition as face_rec
import os
from datetime import datetime

path = 'images'
studentImg = []
studentName = []
myList = os.listdir(path)

for cl in myList:
    curimg = cv2.imread(f'{path}/{cl}')
    studentImg.append(curimg)
    studentName.append(os.path.splitext(cl)[0])

def findEncoding(images):
    imgEncodings = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodeimg = face_rec.face_encodings(img)[0]
        imgEncodings.append(encodeimg)
    return imgEncodings

def MarkAttendence(name):
    with open('attendence.csv', 'r+') as f:
        myDatalist = f.readlines()
        nameList = []
        for line in myDatalist:
            entry = line.split(',')
            nameList.append(entry[0])

        if name not in nameList:
            now = datetime.now()
            timestr = now.strftime('%H:%M')
            f.writelines(f'\n{name}, {timestr}')


EncodeList = findEncoding(studentImg)

vid = cv2.VideoCapture(0)
while True:
    success, frame = vid.read()

    if frame is None:
        break

    Smaller_frames = cv2.resize(frame, (0, 0), None, 0.25, 0.25)

    facesInFrame = face_rec.face_locations(Smaller_frames)
    encodeFacesInFrame = face_rec.face_encodings(Smaller_frames, facesInFrame)

    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame):
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        matchIndex = np.argmin(facedis)

        if matches[matchIndex]:
            name = studentName[matchIndex].upper()
            y1, x2, y2, x1 = faceloc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.rectangle(frame, (x1, y2 - 25), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            MarkAttendence(name)
        else:
            y1, x2, y2, x1 = faceloc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.rectangle(frame, (x1, y2 - 25), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, "Unknown", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            MarkAttendence("Unknown")

    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1)  # Wait for a key event
    if key == 27:  # ASCII code for Esc key
        break

vid.release()  # Release the camera resources
cv2.destroyAllWindows()  # Close all OpenCV windows